# guppy and hugr to qir conversion and submission to H1 and H2


This example shows how to convert guppy to qir which can be submitted directly to H1 and H2 device, emulator and syntax checker.

You need to install hugr-qir, guppy and pytket-quantinuum for this notebook to work.

### Current guppy features that can't be converted:
- loops with condition not known at compiletime
- functions returning qubit arrays
- RNG functions
- dynamic qubit allocation


In [1]:
# You can write your guppy directly in a notebook or in a separate file
import sys
from typing import no_type_check

from guppylang import guppy, qubit
from guppylang.std.builtins import result
from guppylang.std.quantum import h, measure


@guppy
@no_type_check
def main() -> None:
    q0 = qubit()
    q1 = qubit()

    h(q0)
    h(q1)

    b0 = measure(q0)
    b1 = measure(q1)
    b2 = b0 ^ b1

    result("0", b2)

# Convert hugr to qir

By default, the function will automatically check the generated QIR to capture most of the issues that could happen.
This will show an error message with more details about the problem the check can be turned off using the keyword argument `validate_qir = False`

In [2]:
from hugr_qir.hugr_to_qir import hugr_to_qir
from hugr_qir.output import OutputFormat

guppy_qir_bitcode_string = hugr_to_qir(main.compile(), output_format=OutputFormat.BASE64)


In [3]:
# The QIR is normally returned as base64 encoded llvm bitcode.
# To get a human-readable LLVM assemly language string use the keyword argument `output_format = OutputFormat.LLVM_IR`
from hugr_qir.output import OutputFormat


guppy_qir = hugr_to_qir(main.compile(), output_format=OutputFormat.LLVM_IR)
print(guppy_qir)

; ModuleID = 'hugr-qir'
source_filename = "hugr-qir"
target datalayout = "e-m:e-i8:8:32-i16:16:32-i64:64-i128:128-n32:64-S128"
target triple = "aarch64-unknown-linux-gnu"

%Qubit = type opaque
%Result = type opaque

@0 = private unnamed_addr constant [2 x i8] c"0\00", align 1

define void @__hugr__.main.1() local_unnamed_addr #0 {
alloca_block:
  tail call void @__quantum__qis__phasedx__body(double 0x3FF921FB54442D18, double 0xBFF921FB54442D18, %Qubit* null)
  tail call void @__quantum__qis__rz__body(double 0x400921FB54442D18, %Qubit* null)
  tail call void @__quantum__qis__mz__body(%Qubit* null, %Result* null)
  %0 = tail call i1 @__quantum__qis__read_result__body(%Result* null)
  tail call void @__quantum__qis__phasedx__body(double 0x3FF921FB54442D18, double 0xBFF921FB54442D18, %Qubit* nonnull inttoptr (i64 1 to %Qubit*))
  tail call void @__quantum__qis__rz__body(double 0x400921FB54442D18, %Qubit* nonnull inttoptr (i64 1 to %Qubit*))
  tail call void @__quantum__qis__mz__body(%Qubit

In [4]:
# set up a pytket-quantinnum backend
# This required credentials for the device

from pytket.extensions.quantinuum import QuantinuumBackend, Language

backend = QuantinuumBackend(device_name="H1-1SC")
backend.login()


Enter your Quantinuum email:  melf.johannsen@quantinuum.com
Enter your Quantinuum password:  ········


In [5]:
# Submit the qir to the device (which expects the base64 encoded llvm binary)

h = backend.submit_program(Language.QIR, guppy_qir_bitcode_string, n_shots=10)
r = backend.get_result(h)
shots = r.get_shots()
assert len(shots) == 10
print(h)
print(shots)

/run/user/1002/ipykernel_63242/2897221857.py:3: DeprecationWarning: Submission of programs to remote devices from pytket-quantinuum is deprecated, and will not be possible after October 2025. Please use qnexus ( https://docs.quantinuum.com/nexus/index.html ) instead.
  h = backend.submit_program(Language.QIR, guppy_qir_bitcode_string, n_shots=10)


('91540f47d0b44ec4bec6f919e05ea01f', 'null', -1, 'null')
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [6]:
from typing import no_type_check

from guppylang import guppy, qubit
from guppylang.std.builtins import result
from guppylang.std.quantum import h, measure


@guppy
@no_type_check
def main() -> None:
    q0 = qubit()
    q1 = qubit()

    for _ in range(10):
        q3 = qubit()
        h(q3)
        b = measure(q3)
        if b:
            h(q0)

    result("0", measure(q0))
    result("1", measure(q1))



In [7]:
guppy_qir = hugr_to_qir(main.compile(), output_format=OutputFormat.LLVM_IR)
print(guppy_qir)


; ModuleID = 'hugr-qir'
source_filename = "hugr-qir"
target datalayout = "e-m:e-i8:8:32-i16:16:32-i64:64-i128:128-n32:64-S128"
target triple = "aarch64-unknown-linux-gnu"

%Qubit = type opaque
%Result = type opaque

@0 = private unnamed_addr constant [2 x i8] c"0\00", align 1
@1 = private unnamed_addr constant [2 x i8] c"1\00", align 1

define void @__hugr__.main.1() local_unnamed_addr #0 {
alloca_block:
  tail call void @__quantum__qis__phasedx__body(double 0x3FF921FB54442D18, double 0xBFF921FB54442D18, %Qubit* nonnull inttoptr (i64 2 to %Qubit*))
  tail call void @__quantum__qis__rz__body(double 0x400921FB54442D18, %Qubit* nonnull inttoptr (i64 2 to %Qubit*))
  tail call void @__quantum__qis__mz__body(%Qubit* nonnull inttoptr (i64 2 to %Qubit*), %Result* nonnull inttoptr (i64 2 to %Result*))
  %0 = tail call i1 @__quantum__qis__read_result__body(%Result* nonnull inttoptr (i64 2 to %Result*))
  br i1 %0, label %21, label %cond_309_case_0

cond_309_case_0:                              

Constructing a similar example like the one above with a loop which numbers of execution depends on the measurement results inside the loop leads to the generation of QIR which is valid within the standard, but can't be understood by the device.

In [8]:

from typing import no_type_check

from guppylang import guppy, qubit
from guppylang.std.builtins import result
from guppylang.std.quantum import h, measure

@guppy
@no_type_check
def main() -> None:
    q0 = qubit()
    q1 = qubit()
    
    i = 0
    while i < 10:
        q3 = qubit()
        h(q3)
        b = measure(q3)
        if b:
            h(q0)
            i += 1

    result("0", measure(q0))
    result("1", measure(q1))

In [9]:
guppy_qir = hugr_to_qir(main.compile(), output_format=OutputFormat.LLVM_IR, validate_qir=False)
print(guppy_qir)

; ModuleID = 'hugr-qir'
source_filename = "hugr-qir"
target datalayout = "e-m:e-i8:8:32-i16:16:32-i64:64-i128:128-n32:64-S128"
target triple = "aarch64-unknown-linux-gnu"

%Qubit = type opaque
%Result = type opaque

@0 = private unnamed_addr constant [2 x i8] c"0\00", align 1
@1 = private unnamed_addr constant [2 x i8] c"1\00", align 1

define void @__hugr__.main.1() local_unnamed_addr #0 {
alloca_block:
  br label %cond_exit_176

cond_104_case_0:                                  ; preds = %cond_exit_176, %4
  %"65_0.0" = phi i64 [ %5, %4 ], [ %"15_0.0221", %cond_exit_176 ]
  %0 = icmp slt i64 %"65_0.0", 10
  br i1 %0, label %cond_exit_176, label %cond_132_case_1

cond_132_case_1:                                  ; preds = %cond_104_case_0
  tail call void @__quantum__qis__mz__body(%Qubit* null, %Result* null)
  %1 = tail call i1 @__quantum__qis__read_result__body(%Result* null)
  tail call void @__quantum__rt__bool_record_output(i1 %1, i8* getelementptr inbounds ([2 x i8], [2 x i8]* @

In [10]:
# this will fail because of the loop in the generated QIR

try:
    guppy_qir = hugr_to_qir(main.compile(), output_format=OutputFormat.LLVM_IR, validate_qir=True)
    print(guppy_qir)
except Exception as e:
    print('Validation failed as expected:')
    print(e)

Validation failed as expected:
Found loop in CFG containing the block: cond_exit_176


# Submission via Nexus

QIR can be submitted via Nexus, too. It needs to be installed via `pip install qnexus`. This requires a different QIR format for the submission, which can be generated from hugr-qir via the format flag `OutputFormat.BITCODE`.

In [ ]:
import qnexus as qnx

qnx.login()

In [12]:
import datetime

project = qnx.projects.get_or_create(name="QIR-Demonstration")
qnx.context.set_active_project(project)

qir_name = "HUGR-QIR"
jobname_suffix = datetime.datetime.now().strftime("%Y_%m_%d-%H-%M-%S")

In [13]:
# You can write your guppy directly in a notebook or in a separate file
import sys
from typing import no_type_check

from guppylang import guppy, qubit
from guppylang.std.builtins import result
from guppylang.std.quantum import h, measure


@guppy
@no_type_check
def main() -> None:
    q0 = qubit()
    q1 = qubit()

    h(q0)
    h(q1)

    b0 = measure(q0)
    b1 = measure(q1)
    b2 = b0 ^ b1

    result("0", b2)

In [14]:
from hugr_qir.hugr_to_qir import hugr_to_qir

guppy_qir_bitcode = hugr_to_qir(main.compile(), output_format=OutputFormat.BITCODE)

In [15]:
qir_program_ref = qnx.qir.upload(qir=guppy_qir_bitcode, name=qir_name, project=project)

In [16]:
# Run on the H1-1 Syntax checker
device_name = "H1-1SC"

qnx.context.set_active_project(project)
config = qnx.QuantinuumConfig(device_name=device_name)

job_name = f"execution-job-qir-{qir_name}-{device_name}-{jobname_suffix}"
ref_execute_job = qnx.start_execute_job(
    programs=[qir_program_ref],
    n_shots=[10],
    backend_config=config,
    name=job_name,
)

qnx.jobs.wait_for(ref_execute_job)

JobStatus(status=<JobStatusEnum.COMPLETED: 'COMPLETED'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2025, 9, 4, 9, 26, 55, 990444, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2025, 9, 4, 9, 26, 27, 386952, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [17]:
qir_result = qnx.jobs.results(ref_execute_job)[0].download_result()
qir_result.get_counts()

Counter({(0,): 10})